# Delaunay-Watershed for geometry reconstruction: cell aggregate example

In this notebook:
- We show how to construct a multimaterial mesh from segmentation masks of cells in an aggregate.
- We show how to easily visualize the mesh.

# 0) Install Delaunay-Watershed

We install the viewing option from Delaunay-Watershed, based on Napari and Polyscope, to visualize the mesh later.

In [ ]:
pip install "delaunay-watershed-3d[viewing]"

## 1) Compute multimaterial mesh with Delaunay Watershed

A mesh is created via a reconstruction object. The creation of this object makes all necessary computations and the mesh can be used just after that:

In [ ]:
import skimage.io as io
from dw3d import GeometryReconstruction3D

dist = 3

image_path = "../data/Images/1.tif"
image = io.imread(image_path)
reconstruction = GeometryReconstruction3D(image, min_dist = dist, print_info=True)

Now the mesh is available, through the program for example:

In [ ]:
# The multimaterial mesh is:
# points: an array of 3D points (geometry)
# triangles: an array of triangles (topology)
# labels: an array of materials on each side of a triangle.
points, triangles, labels = reconstruction.mesh

print(f"{points=}")
print(f"{triangles=}")
print(f"{labels=}")

By default, the mesh has the same dimension as the image in pixels. The dimensions of the mesh can be set easily to match the real sample size:

In [ ]:
known_xy_size_of_pixel = 0.1  # say, in micrometers for example
known_z_size_of_pixel = 1.5
reconstruction.set_pixel_size(xy_pixel_size=known_xy_size_of_pixel, z_pixel_size=known_z_size_of_pixel)
# Before, reconstruction.mesh was of size image.shape
# Now, reconstruction.mesh as the real size of the sample, here it is between 0 and some size in micrometers.

reconstruction.center_around_zero()
# Now reconstruction.mesh is also centered around 0.


Alternatively, you can set a global minimum and maximum for the size of the mesh:

In [ ]:
reconstruction.set_global_min_max(global_min=-1, global_max=1)
# Now reconstruction.mesh is between -1 and 1.

The mesh can be saved easily for future use and analysis (with foambryo for example):

In [ ]:
# The multimaterial mesh can be saved in a .vtk or .rec format. In binary or ASCII.
reconstruction.save_to_rec_mesh("mesh_1.rec", binary_mode=False)
reconstruction.save_to_vtk_mesh("mesh_1.vtk", binary_mode=True)
print("Mesh saved.")

## 2) Visualize mesh with our Polyscope viewer.

In [4]:
from dw3d.viewing import plot_cells_polyscope

plot_cells_polyscope(reconstruction)

# sometimes, it's better to scatter cells to understand the geometry
# plot_cells_polyscope(reconstruction, view="Scattered", scattering_coeff=0.5)

# 3) Understand the construction process with Napari

In [ ]:
from dw3d.viewing import plot_in_napari

v = plot_in_napari(reconstruction, add_mesh=True)